In [22]:
# init
from pyquda.utils import gauge_utils
from pyquda.field import LatticeFermion
from pyquda import core, pyqcu, mpi
import os
import sys
from time import perf_counter
import cupy as cp
test_dir = os.path.dirname(os.path.abspath("./"))
sys.path.insert(0, os.path.join(test_dir, ".."))
os.environ["QUDA_RESOURCE_PATH"] = ".cache"
latt_size = [16, 16, 16, 32]
grid_size = [1, 1, 1, 1]
Lx, Ly, Lz, Lt = latt_size
Nd, Ns, Nc = 4, 4, 3
Gx, Gy, Gz, Gt = grid_size
latt_size = [Lx//Gx, Ly//Gy, Lz//Gz, Lt//Gt]
Lx, Ly, Lz, Lt = latt_size
Vol = Lx * Ly * Lz * Lt
mpi.init(grid_size)
latt_tmp0 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
latt_tmp1 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
latt_shape = (Lt, Lz, Ly, Lx//2, Ns, Nc)
param = pyqcu.QcuParam()
param.lattice_size = latt_size
_dslash = core.getDslash(latt_size, -3.5, 0, 0, anti_periodic_t=False)
kappa = 0.125
U = gauge_utils.gaussGauge(latt_size, 0)
_dslash.loadGauge(U)

Creating Gaussian distrbuted Lie group field with sigma = 1.000000e-01


In [23]:
# give ans first
ans_e = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_e
ans_o = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_o

In [24]:
# give x_o, b__o, r, r_tilde, p, v, s, t
x_o = cp.random.random(latt_shape) + 1j * cp.random.random(latt_shape)  # x_o
b_e = cp.zeros(latt_shape, cp.complex128)
b_o = cp.zeros(latt_shape, cp.complex128)
b__o = cp.zeros(latt_shape, cp.complex128)
r = cp.zeros(latt_shape, cp.complex128)
r_tilde = cp.zeros(latt_shape, cp.complex128)
p = cp.zeros(latt_shape, cp.complex128)
s = cp.zeros(latt_shape, cp.complex128)
v = cp.zeros(latt_shape, cp.complex128)
t = cp.zeros(latt_shape, cp.complex128)
# give r_norm2, MAX_ITER, TOL, rho_prev, rho, alpha, omega, beta, kappa
r_norm2 = 0
MAX_ITER = 1e3
TOL = 1e-16
rho_prev = 1
rho = 0
alpha = 1
omega = 1
beta = 0
kappa = 0.125

In [25]:
# define dslash
def dslash_eo(src_o):
    latt_tmp1.data[1, :] = src_o
    pyqcu.dslashQcu(latt_tmp0.even_ptr, latt_tmp1.odd_ptr,
                    U.data_ptr, param, 0)  # D_eo
    return latt_tmp0.data[0, :]


def dslash_oe(src_e):
    latt_tmp1.data[0, :] = src_e
    pyqcu.dslashQcu(latt_tmp0.odd_ptr, latt_tmp1.even_ptr,
                    U.data_ptr, param, 1)  # D_oe
    return latt_tmp0.data[1, :]


def dslash(src_o):
    return src_o-kappa**2*dslash_oe(dslash_eo(src_o))

In [26]:
# give b'_o(b__0)
b__o = (ans_o-kappa*dslash_oe(ans_e))+kappa*dslash_oe((ans_e-kappa*dslash_eo(ans_o)))

In [27]:
def dot(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.inner(a.flatten().conjugate(), b.flatten())


def diff(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.linalg.norm(a - b) / cp.linalg.norm(b)

In [28]:
# bistabcg
r = b__o - dslash(x_o)
r_tilde = r

In [29]:
t0 = perf_counter()
for i in range(1, int(MAX_ITER)):
    # print("## rho:", rho)
    rho = dot(r_tilde, r)
    # print("## beta:", beta)
    beta = (rho / rho_prev) * (alpha / omega)
    p = r + (p - v * omega) * beta
    # v = A * p
    v = dslash(p)
    # print("## alpha:", alpha)
    alpha = rho / dot(r_tilde, v)
    s = r - v * alpha
    # t = A * s
    t = dslash(s)
    # print("## omega:", omega)
    omega = dot(t, s) / dot(t, t)
    x_o = x_o + p * alpha + s * omega
    r = s - t * omega
    r_norm2 = cp.linalg.norm(r)
    print("##{}# r_norm2:{}".format(i, r_norm2))
    # break
    if (r_norm2 < TOL or i == MAX_ITER - 1):
        print("### turns:", i)
        break
    rho_prev = rho
t1 = perf_counter()

##1# r_norm2:52.748147431216736
##2# r_norm2:22.918872979543274
##3# r_norm2:28.983765420049437
##4# r_norm2:18.099250517729356
##5# r_norm2:6.385697157439168
##6# r_norm2:3.4927236617176907
##7# r_norm2:2.2766110562870465
##8# r_norm2:1.9301638865756303
##9# r_norm2:0.7293340106966993
##10# r_norm2:0.5367628751381203
##11# r_norm2:0.5554237186491864
##12# r_norm2:0.7997276166759395
##13# r_norm2:0.35409999633727823
##14# r_norm2:0.1747307974336775
##15# r_norm2:0.12377004439435402
##16# r_norm2:0.19693701048484513
##17# r_norm2:0.15653632286859534
##18# r_norm2:0.41728770504221036
##19# r_norm2:0.12235594562539182
##20# r_norm2:0.10531996667562649
##21# r_norm2:0.1905186709240021
##22# r_norm2:0.051113992772515444
##23# r_norm2:0.04439654535596471
##24# r_norm2:0.03867357726182905
##25# r_norm2:0.0351525301351579
##26# r_norm2:0.03370837306305843
##27# r_norm2:0.02035147974290728
##28# r_norm2:0.01705682635474654
##29# r_norm2:0.014333901667123157
##30# r_norm2:0.013504169038219539
##

In [30]:
print("### ans_o = ", ans_o[0, 0, 0, 0, :, :])
print("### x_o = ", x_o[0, 0, 0, 0, :, :])
print('## difference: ', diff(x_o, ans_o))
print(f'Quda dslash: {t1 - t0} sec')

### ans_o =  [[0.09951438+0.41070745j 0.46854031+0.21225794j 0.72409208+0.38866208j]
 [0.39502885+0.05041868j 0.1659288 +0.82601784j 0.70421626+0.1569881j ]
 [0.95061153+0.37311491j 0.61962787+0.50843883j 0.6060668 +0.63786447j]
 [0.17985272+0.44894298j 0.39347176+0.2624171j  0.94261466+0.86165941j]]
### x_o =  [[0.09951438+0.41070745j 0.46854031+0.21225794j 0.72409208+0.38866208j]
 [0.39502885+0.05041868j 0.1659288 +0.82601784j 0.70421626+0.1569881j ]
 [0.95061153+0.37311491j 0.61962787+0.50843883j 0.6060668 +0.63786447j]
 [0.17985272+0.44894298j 0.39347176+0.2624171j  0.94261466+0.86165941j]]
## difference:  8.717502337584867e-16
Quda dslash: 5.523802263999642 sec


In [31]:
x_e = ans_e-kappa*dslash_eo(ans_o)+kappa * \
    dslash_eo(x_o)  # equal to x_o == ans_o
print(diff(x_e, ans_e))

4.860580518759557e-16
